In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load the data
data = pd.read_csv("train.csv", low_memory=False)
data.drop_duplicates() # remove duplicates

,Characteristics.LotFeatures,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.features_reso.results,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,...,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.Heating,Structure.LivingArea,Structure.NewConstructionYN,Structure.ParkingFeatures,Structure.Rooms.RoomsTotal,Structure.YearBuilt,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,3200.0,NaN,NaN,NaN,NaN,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,0.0,commr,NaN
1,NaN,NaN,NaN,3.8,3.7,3.8,3.8,"['Appliances.GasRange', 'Appliances.Range', 'A...",NaN,4.2,...,NaN,NaN,['natural gas'],3175.0,False,['off alley'],6.0,NaN,NaN,NaN
2,NaN,18750.0,NaN,NaN,NaN,NaN,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,1926.0,commr,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...",NaN,NaN,...,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']",NaN,NaN,3.6,NaN,NaN,3.6,"['Appliances.Refrigerator', 'Appliances.Microw...",NaN,4.2,...,NaN,NaN,['other'],0.0,False,"['unassigned', 'off street']",4.0,2006.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,NaN,NaN,NaN,3.8,NaN,NaN,3.8,"['Levels.One', 'ParkingFeatures.Attached', 'Pa...",NaN,4.1,...,NaN,2.0,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN
107433,NaN,NaN,2.8,NaN,3.6,3.1,3.1,"['Appliances.Dishwasher', 'Appliances.Stainles...",3.2,NaN,...,0.0,NaN,"['natural gas', 'forced air']",0.0,False,"['assigned', 'off street']",9.0,1963.0,NaN,NaN
107434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['ExteriorFeatures.PrivateYard', 'ExteriorFeat...",NaN,NaN,...,NaN,2.5,NaN,NaN,False,['driveway'],NaN,NaN,NaN,NaN
107435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['ParkingFeatures.ParkingLot'],NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# DATA CLEANING OF STORIES


# Map the 'ImageData.style.stories.summary.label' column to numeric values
story_mapping = {
    '1_story': 1,
    '1.5_stories': 1.5,
    '2_stories': 2,
    '2.5_stories': 2.5,
    '3_stories_or_more': 3
}


# print(data['ImageData.style.stories.summary.label'].unique())
data['ImageData.style.stories.summary.label'] = data['ImageData.style.stories.summary.label'].map(story_mapping)
print(data['ImageData.style.stories.summary.label'].unique())

[nan 2.  1.  1.5 2.5 3. ]


In [4]:
data['Structure.YearBuilt'].unique()

array([   0.,   nan, 1926., 2006., 1967., 1990., 1976., 1940., 2007.,
       1969., 1928., 1946., 1930., 1965., 1927., 1968., 1958., 2022.,
       1895., 1952., 1948., 1978., 1899., 1987., 2023., 1985., 2003.,
       1960., 1910., 1925., 1999., 1959., 2021., 1950., 1894., 1905.,
       1964., 1935., 1896., 1920., 1880., 2009., 1923., 1901., 1988.,
       1900., 1907., 1911., 1889., 2000., 1974., 1970., 1924., 1893.,
       1961., 1914., 1954., 2004., 1909., 1939., 1903., 1955., 1912.,
       2019., 1918., 1890., 1951., 2005., 1971., 1995., 1908., 1984.,
       1932., 2008., 2001., 1887., 1997., 1962., 1885., 1874., 1931.,
       2024., 2012., 1953., 1994., 1941., 1947., 1957., 1863., 1949.,
       1917., 1956., 1986., 1898., 1992., 1929., 1973., 1972., 1884.,
       1996., 1938., 1855., 1998., 1852., 1888., 1892., 1913., 1906.,
       1915., 1975., 1989., 1963., 1919., 1856., 1943., 2011., 2013.,
       2020., 1981., 1904., 2002., 1982., 1979., 1977., 1980., 1922.,
       1875., 2018.,

In [5]:
# # replace edge case 
# data['Structure.YearBuilt'] = data['Structure.YearBuilt'].replace(190., np.nan)

# # convert 0 to nan on Structure.YearBuilt to remove 0 values
# data['Structure.YearBuilt'] = data['Structure.YearBuilt'].replace(0, np.nan)


In [6]:
#data segmentation
columnes_numeriques = data.select_dtypes(include=['number'])  # Selecciona columnes numèriques
columnes_categoriques = data.select_dtypes(exclude=['number'])  # Selecciona columnes no numèriques (categòriques)

In [7]:

# Treballar amb cada columna numèrica una a una
for columna in columnes_numeriques.columns:
    # Calcular Q1, Q3 i IQR per a la columna actual (ignorant els NaN)
    Q1 = columnes_numeriques[columna].quantile(0.1)
    Q3 = columnes_numeriques[columna].quantile(0.9)
    IQR = Q3 - Q1
    
    # Límits per identificar outliers
    lower_bound = Q1 - 2 * IQR
    upper_bound = Q3 + 2 * IQR
    
    # Filtrar files dins dels límits, mantenint files amb NaN
    mask = (columnes_numeriques[columna].isna()) | \
        ((columnes_numeriques[columna] >= lower_bound) & (columnes_numeriques[columna] <= upper_bound))
    
    columnes_numeriques = columnes_numeriques[mask]

columnes_numeriques

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
1,NaN,NaN,3.8,3.7,3.8,3.8,NaN,4.2,4.0,4.0,...,2.0,1.0,1.0,NaN,NaN,NaN,NaN,3175.0,6.0,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,12144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1967.0
6,NaN,NaN,3.0,NaN,NaN,3.0,NaN,3.0,NaN,NaN,...,7.0,2.0,5.0,4384.0,NaN,2.0,5.0,787.0,10.0,1990.0
7,11548.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,NaN,NaN,3.8,NaN,NaN,3.8,NaN,4.1,NaN,NaN,...,1.0,0.0,2.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN
107433,NaN,2.8,NaN,3.6,3.1,3.1,3.2,NaN,3.1,2.7,...,3.0,0.0,4.0,NaN,NaN,0.0,NaN,0.0,9.0,1963.0
107434,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,1.0,5.0,NaN,NaN,NaN,2.5,NaN,NaN,NaN
107435,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# canviar els NaN per la mitjana de la columna
columnes_numeriques = columnes_numeriques.apply(lambda col: col.fillna(col.mean(skipna=True)) if col.dtype in ['float64', 'int64'] else col)
columnes_numeriques

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
1,18697.555959,3.119845,3.800000,3.700000,3.80000,3.800000,3.318735,4.200000,4.000000,4.000000,...,2.000000,1.000000,1.000000,683.820481,646.035198,0.91617,1.894976,3175.000000,6.000000,1970.084268
3,18697.555959,3.119845,3.310584,3.214658,3.12045,3.186064,3.318735,3.506304,3.114642,3.218744,...,1.894617,0.433305,3.054186,683.820481,646.035198,0.91617,1.894976,1699.664821,7.086452,1970.084268
5,12144.000000,3.119845,3.310584,3.214658,3.12045,3.186064,3.318735,3.506304,3.114642,3.218744,...,1.894617,0.433305,3.054186,683.820481,646.035198,0.00000,1.894976,1699.664821,7.086452,1967.000000
6,18697.555959,3.119845,3.000000,3.214658,3.12045,3.000000,3.318735,3.000000,3.114642,3.218744,...,7.000000,2.000000,5.000000,4384.000000,646.035198,2.00000,5.000000,787.000000,10.000000,1990.000000
7,11548.000000,3.119845,3.310584,3.214658,3.12045,3.186064,3.318735,3.506304,3.114642,3.218744,...,1.894617,0.433305,3.054186,683.820481,646.035198,0.91617,1.894976,1699.664821,7.086452,1976.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107432,18697.555959,3.119845,3.800000,3.214658,3.12045,3.800000,3.318735,4.100000,3.114642,3.218744,...,1.000000,0.000000,2.000000,683.820481,646.035198,0.91617,2.000000,1699.664821,7.086452,1970.084268
107433,18697.555959,2.800000,3.310584,3.600000,3.10000,3.100000,3.200000,3.506304,3.100000,2.700000,...,3.000000,0.000000,4.000000,683.820481,646.035198,0.00000,1.894976,0.000000,9.000000,1963.000000
107434,18697.555959,3.119845,3.310584,3.214658,3.12045,3.186064,3.318735,3.506304,3.114642,3.218744,...,3.000000,1.000000,5.000000,683.820481,646.035198,0.91617,2.500000,1699.664821,7.086452,1970.084268
107435,18697.555959,3.119845,3.310584,3.214658,3.12045,3.186064,3.318735,3.506304,3.114642,3.218744,...,1.894617,0.433305,3.054186,683.820481,646.035198,0.91617,1.894976,1699.664821,7.086452,1970.084268


In [9]:
columnes_numeriques['Characteristics.LotSizeSquareFeet'].max()

174600.0

In [10]:
# normalize all numeric columns
scaler = MinMaxScaler()
columnes_numeriques = pd.DataFrame(scaler.fit_transform(columnes_numeriques), columns=columnes_numeriques.columns)
columnes_numeriques

,Characteristics.LotSizeSquareFeet,ImageData.c1c6.summary.bathroom,ImageData.c1c6.summary.exterior,ImageData.c1c6.summary.interior,ImageData.c1c6.summary.kitchen,ImageData.c1c6.summary.property,ImageData.q1q6.summary.bathroom,ImageData.q1q6.summary.exterior,ImageData.q1q6.summary.interior,ImageData.q1q6.summary.kitchen,...,Structure.BathroomsFull,Structure.BathroomsHalf,Structure.BedroomsTotal,Structure.BelowGradeFinishedArea,Structure.BelowGradeUnfinishedArea,Structure.FireplacesTotal,Structure.GarageSpaces,Structure.LivingArea,Structure.Rooms.RoomsTotal,Structure.YearBuilt
0,0.107088,0.423969,0.560000,0.540000,0.56000,0.560000,0.565545,0.727273,0.731707,0.750000,...,0.285714,0.333333,0.125000,0.155981,0.153161,0.152695,0.270711,0.423333,0.272727,0.782598
1,0.107088,0.423969,0.462117,0.442932,0.42409,0.437213,0.565545,0.569614,0.515766,0.554686,...,0.270660,0.144435,0.381773,0.155981,0.153161,0.152695,0.270711,0.226622,0.322111,0.782598
2,0.069553,0.423969,0.462117,0.442932,0.42409,0.437213,0.565545,0.569614,0.515766,0.554686,...,0.270660,0.144435,0.381773,0.155981,0.153161,0.000000,0.270711,0.226622,0.322111,0.770161
3,0.107088,0.423969,0.400000,0.442932,0.42409,0.400000,0.565545,0.454545,0.515766,0.554686,...,1.000000,0.666667,0.625000,1.000000,0.153161,0.333333,0.714286,0.104933,0.454545,0.862903
4,0.066140,0.423969,0.462117,0.442932,0.42409,0.437213,0.565545,0.569614,0.515766,0.554686,...,0.270660,0.144435,0.381773,0.155981,0.153161,0.152695,0.270711,0.226622,0.322111,0.806452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105099,0.107088,0.423969,0.560000,0.442932,0.42409,0.560000,0.565545,0.704545,0.515766,0.554686,...,0.142857,0.000000,0.250000,0.155981,0.153161,0.152695,0.285714,0.226622,0.322111,0.782598
105100,0.107088,0.360000,0.462117,0.520000,0.42000,0.420000,0.536585,0.569614,0.512195,0.425000,...,0.428571,0.000000,0.500000,0.155981,0.153161,0.000000,0.270711,0.000000,0.409091,0.754032
105101,0.107088,0.423969,0.462117,0.442932,0.42409,0.437213,0.565545,0.569614,0.515766,0.554686,...,0.428571,0.333333,0.625000,0.155981,0.153161,0.152695,0.357143,0.226622,0.322111,0.782598
105102,0.107088,0.423969,0.462117,0.442932,0.42409,0.437213,0.565545,0.569614,0.515766,0.554686,...,0.270660,0.144435,0.381773,0.155981,0.153161,0.152695,0.270711,0.226622,0.322111,0.782598


In [11]:
columnes_categoriques.head()

,Characteristics.LotFeatures,ImageData.features_reso.results,ImageData.room_type_reso.results,ImageData.style.exterior.summary.label,Listing.Dates.CloseDate,Listing.ListingId,Location.Address.CensusBlock,Location.Address.CensusTract,Location.Address.City,Location.Address.CountyOrParish,...,Location.Area.SubdivisionName,Location.School.HighSchoolDistrict,Property.PropertyType,Structure.Basement,Structure.Cooling,Structure.Heating,Structure.NewConstructionYN,Structure.ParkingFeatures,Tax.Zoning,UnitTypes.UnitTypeType
0,NaN,"['Cooling.CeilingFans', 'Flooring.Carpet', 'Fl...","['Other', 'FrontOfStructure', 'Basement', 'Sta...",NaN,2023-12-22T00:00:00,mrd09114306,17-195-300-1004,17-195-300,morrison,whiteside,...,NaN,NaN,commercial sale,NaN,['central air'],NaN,False,NaN,commr,NaN
1,NaN,"['Appliances.GasRange', 'Appliances.Range', 'A...","['SideOfStructure', 'Other', 'Kitchen', 'Office']",NaN,2023-09-19T00:00:00,mrd09478582,17-97-865400-2007,17-97-865400,highland park,lake,...,NaN,112,residential,['partial'],['window/wall unit - 1'],['natural gas'],False,['off alley'],NaN,NaN
2,NaN,"['Flooring.Carpet', 'ParkingFeatures.Garage', ...","['Community', 'FrontOfStructure', 'Other', 'Ga...",bungalow,2023-12-04T00:00:00,mrd09595738,17-31-210900-2014,17-31-210900,chicago,cook,...,NaN,NaN,commercial sale,NaN,['central air'],NaN,False,NaN,commr,NaN
3,NaN,"['CommunityFeatures.Lake', 'WaterfrontFeatures...","['Yard', 'View']",NaN,2023-08-01T00:00:00,mrd09604772,NaN,NaN,wilmington,will,...,NaN,209u,farm,NaN,NaN,NaN,False,NaN,agric,NaN
4,"['horses allowed', 'paddock', 'pond(s)']","['Appliances.Refrigerator', 'Appliances.Microw...","['FrontOfStructure', 'BackOfStructure', 'Commu...",ranch,2023-09-20T00:00:00,mrd09795957,17-89-852403-1029,17-89-852403,maple park,kane,...,NaN,301,residential,['full'],"['partial', 'none']",['other'],False,"['unassigned', 'off street']",NaN,NaN


In [12]:
columnes_numeriques.to_csv('columnes_numeriques.csv', index=False)
columnes_categoriques.to_csv('columnes_categoriques.csv', index=False)